In [1]:
import json

from pathlib import Path
from IPython.display import HTML, display

from joint import data

# put there the path to the results you want to show
FOLDER_3_WEIGHTED_EMB='joint/results/viz/eval_loss_both_slottype_full_we_large_recurrent_cell_lstm_attention_both_three_stages_intentextraction_word-emb___hyper:LABEL_EMB_SIZE=64,LSTM_SIZE=128,BATCH_SIZE=2,MAX_EPOCHS=100/huric_eb/modern'
FOLDER_3_BI_RNN='joint/results/viz/eval_loss_both_slottype_full_we_large_recurrent_cell_lstm_attention_both_three_stages___hyper:LABEL_EMB_SIZE=64,LSTM_SIZE=128,BATCH_SIZE=2,MAX_EPOCHS=100/huric_eb/modern'
FOLDER_2_WEIGHTED_EMB='joint/results/viz/eval_loss_both_slottype_full_we_large_recurrent_cell_lstm_attention_both_intentextraction_word-emb___hyper:LABEL_EMB_SIZE=64,LSTM_SIZE=128,BATCH_SIZE=2,MAX_EPOCHS=100/huric_eb/modern'
FOLDER_2_BI_RNN='joint/results/viz/eval_loss_both_slottype_full_we_large_recurrent_cell_lstm_attention_both___hyper:LABEL_EMB_SIZE=64,LSTM_SIZE=128,BATCH_SIZE=2,MAX_EPOCHS=100/huric_eb/modern'

def load_json(folder, epoch=99):
    json_location = Path(folder) / 'json' / 'epoch_{}'.format(epoch) / 'prediction_fold_full.json'
    with open(json_location) as f:
        content = json.load(f)
    return content['samples']

def get_intent_attention_arrays(sentence_data):
    """returns true (lexical units) and pred attentions"""
    lexical_units = [0] * sentence_data['length']
    for lu in sentence_data['lexical_unit_ids']:
        lexical_units[lu - sentence_data['start_token_id']] = 1
    return lexical_units, sentence_data['intent_attentions']

#def get_bd_attention_arrays(sentence_data):
#    return sentence_data['words'], sentence_data['slots_pred']
   
def get_color(value):
    """Colors a cell by the value provided [0,1] if value is number, otherwise white"""
    
    if isinstance(value, str):
        return 'rgb(255,255,255)'
        print('str', value)
    else:
        v = "%.4f" %  (255 - value * 255)
        return 'rgb({}, 255,{})'.format(v, v)
    
def float_to_str(value):
    if isinstance(value, str):
        return value
    else:
        return "%.4f" % float(value)
    
def display_sequences(row_names, sequences):
    html_str = '<table><tr>{}</tr></table>'.format(
        '</tr><tr>'.join(
            '<td><b>{}</b></td>'.format(row_name) +
            ''.join(['<td style="background-color: {};">{}</td>'.format(get_color(value), float_to_str(value)) for value in row])
            for row_name, row in zip(row_names, sequences)
        )
    )
    display(HTML(html_str))

def display_align(row_labels, col_labels, values):
    html_str = '<table><tr>{}</tr><tr>{}</tr></table>'.format(
        # the headers
        ''.join(['<td>{}</td>'.format(t) for t in [''] + row_labels]),
        # the body
        '</tr><tr>'.join(
            '<td>{}</td>'.format(col_labels[row_idx]) +
            ''.join(['<td style="background-color: {};">{}</td>'.format(get_color(value), float_to_str(value)) for value in row])
            for row_idx, row in enumerate(values)
        )
    )
    display(HTML(html_str))

def display_all(samples):
    for idx, sample in enumerate(samples):
        print('true:', sample['intent_true'], 'pred:', sample['intent_pred'])
        attentions_true, attentions_pred = get_intent_attention_arrays(sample)
        display_sequences(['words', 'lexical_unit','attention_intent', 'slots_true', 'slots_pred'],
                          [sample['words'], attentions_true, attentions_pred, sample['slots_true'], sample['slots_pred']])
        bd = data.slots_to_iob_only(sample['slots_pred'])
        ac = data.slots_to_types_only(sample['slots_pred'])
        display_align(sample['words'], bd, sample['bd_attentions'])
        bd_and_words = ['{}+{}'.format(b, w) for b, w in zip(bd, sample['words'])]
        display_align(bd_and_words, ac, sample['ac_attentions'])
        display_align(sample['words'], sample['slots_pred'], sample['slots_attentions'])
        # don't make the browser crash with all those tables
        if idx >= 50:
            return


In [2]:
data_weighted_emb = load_json(FOLDER_3_WEIGHTED_EMB)
display_all(data_weighted_emb)

true: Inspecting pred: Inspecting


words,check,the,toilet,paper
lexical_unit,1.0000,0.0000,0.0000,0.0000
attention_intent,0.9959,0.0000,0.0002,0.0000
slots_true,O,B-Ground,I-Ground,I-Ground
slots_pred,O,B-Phenomenon,I-Phenomenon,I-Phenomenon


,check,the,toilet,paper
O,0.2838,0.0968,0.2879,0.3314
B-_,0.0624,0.0183,0.0331,0.8863
I-_,0.0106,0.0072,0.0125,0.9696
I-_,0.0029,0.0043,0.0081,0.9847


,O+check,B-_+the,I-_+toilet,I-_+paper
O,0.2813,0.1087,0.1528,0.4572
Phenomenon,0.0987,0.0710,0.1053,0.7250
Phenomenon,0.0941,0.0755,0.1023,0.7281
Phenomenon,0.0862,0.0698,0.1016,0.7425


,check,the,toilet,paper
O,0.0000,0.0000,0.0000,0.0000
B-Phenomenon,0.0000,0.0000,0.0000,0.0000
I-Phenomenon,0.0000,0.0000,0.0000,0.0000
I-Phenomenon,0.0000,0.0000,0.0000,0.0000


...

true: Searching pred: Taking


words,and,find,the,black,pen
lexical_unit,0.0000,1.0000,0.0000,0.0000,0.0000
attention_intent,0.0052,0.0000,0.0000,0.9783,0.0035
slots_true,O,O,B-Phenomenon,I-Phenomenon,I-Phenomenon
slots_pred,O,O,B-Phenomenon,I-Phenomenon,I-Phenomenon


,and,find,the,black,pen
O,0.0493,0.2259,0.1659,0.1273,0.4317
O,0.0701,0.2948,0.2817,0.1916,0.1618
B-_,0.0255,0.3593,0.0800,0.0763,0.4588
I-_,0.0093,0.0563,0.0248,0.0302,0.8794
I-_,0.0061,0.0362,0.0200,0.0238,0.9140


,O+and,O+find,B-_+the,I-_+black,I-_+pen
O,0.1634,0.2418,0.1143,0.1466,0.3340
O,0.1720,0.2788,0.1129,0.1600,0.2763
Phenomenon,0.1019,0.1210,0.0862,0.1205,0.5705
Phenomenon,0.1037,0.1114,0.1003,0.1035,0.5811
Phenomenon,0.0856,0.1071,0.0936,0.1034,0.6103


,and,find,the,black,pen
O,0.0000,0.0000,0.0000,0.0000,0.0000
O,0.0000,0.0000,0.0000,0.0000,0.0000
B-Phenomenon,0.0000,0.0000,0.0000,0.0000,0.0000
I-Phenomenon,0.0000,0.0000,0.0000,0.0000,0.0000
I-Phenomenon,0.0000,0.0000,0.0000,0.0000,0.0000


In [3]:
data_weighted_emb = load_json(FOLDER_3_BI_RNN)
display_all(data_weighted_emb)

true: Inspecting pred: Inspecting


words,check,the,toilet,paper
lexical_unit,1.0000,0.0000,0.0000,0.0000
attention_intent,0.0000,0.7257,0.2281,0.0461
slots_true,O,B-Ground,I-Ground,I-Ground
slots_pred,O,B-Theme,I-Theme,I-Theme


,check,the,toilet,paper
O,0.9940,0.0025,0.0023,0.0012
B-_,0.7897,0.1109,0.0749,0.0245
I-_,0.3002,0.3825,0.2064,0.1109
I-_,0.4173,0.3193,0.1714,0.0920


,O+check,B-_+the,I-_+toilet,I-_+paper
O,0.7660,0.0160,0.0442,0.1737
Theme,0.4090,0.0741,0.1876,0.3293
Theme,0.5896,0.0404,0.0673,0.3027
Theme,0.6213,0.0366,0.0648,0.2773


,check,the,toilet,paper
O,0.0000,0.0000,0.0000,0.0000
B-Theme,0.0000,0.0000,0.0000,0.0000
I-Theme,0.0000,0.0000,0.0000,0.0000
I-Theme,0.0000,0.0000,0.0000,0.0000


...

true: Searching pred: Searching


words,and,find,the,black,pen
lexical_unit,0.0000,1.0000,0.0000,0.0000,0.0000
attention_intent,0.0000,0.0006,0.7356,0.2591,0.0047
slots_true,O,O,B-Phenomenon,I-Phenomenon,I-Phenomenon
slots_pred,O,O,B-Phenomenon,I-Phenomenon,I-Phenomenon


,and,find,the,black,pen
O,0.0038,0.9921,0.0033,0.0005,0.0003
O,0.0021,0.9952,0.0021,0.0004,0.0003
B-_,0.0055,0.7235,0.2215,0.0414,0.0081
I-_,0.0081,0.6197,0.2813,0.0638,0.0271
I-_,0.0076,0.6257,0.2748,0.0536,0.0384


,O+and,O+find,B-_+the,I-_+black,I-_+pen
O,0.2209,0.7301,0.0253,0.0112,0.0125
O,0.2277,0.7096,0.0315,0.0140,0.0171
Phenomenon,0.2425,0.4667,0.1155,0.0883,0.0869
Phenomenon,0.3070,0.5607,0.0564,0.0328,0.0432
Phenomenon,0.3549,0.5317,0.0468,0.0276,0.0389


,and,find,the,black,pen
O,0.0000,0.0000,0.0000,0.0000,0.0000
O,0.0000,0.0000,0.0000,0.0000,0.0000
B-Phenomenon,0.0000,0.0000,0.0000,0.0000,0.0000
I-Phenomenon,0.0000,0.0000,0.0000,0.0000,0.0000
I-Phenomenon,0.0000,0.0000,0.0000,0.0000,0.0000


In [4]:
data_weighted_emb = load_json(FOLDER_2_WEIGHTED_EMB)
display_all(data_weighted_emb)

true: Inspecting pred: Inspecting


words,check,the,toilet,paper
lexical_unit,1.0000,0.0000,0.0000,0.0000
attention_intent,1.0000,0.0000,0.0000,0.0000
slots_true,O,B-Ground,I-Ground,I-Ground
slots_pred,O,B-Phenomenon,I-Phenomenon,I-Phenomenon


,check,the,toilet,paper
O,0.0000,0.0000,0.0000,0.0000
B-_,0.0000,0.0000,0.0000,0.0000
I-_,0.0000,0.0000,0.0000,0.0000
I-_,0.0000,0.0000,0.0000,0.0000


,O+check,B-_+the,I-_+toilet,I-_+paper
O,0.0000,0.0000,0.0000,0.0000
Phenomenon,0.0000,0.0000,0.0000,0.0000
Phenomenon,0.0000,0.0000,0.0000,0.0000
Phenomenon,0.0000,0.0000,0.0000,0.0000


,check,the,toilet,paper
O,0.9755,0.0062,0.0172,0.0012
B-Phenomenon,0.8034,0.0598,0.1279,0.0088
I-Phenomenon,0.1829,0.1302,0.2459,0.4410
I-Phenomenon,0.1649,0.1285,0.1862,0.5204


...

true: Searching pred: Searching


words,and,find,the,black,pen
lexical_unit,0.0000,1.0000,0.0000,0.0000,0.0000
attention_intent,0.0000,1.0000,0.0000,0.0000,0.0000
slots_true,O,O,B-Phenomenon,I-Phenomenon,I-Phenomenon
slots_pred,O,O,B-Phenomenon,I-Phenomenon,I-Phenomenon


,and,find,the,black,pen
O,0.0000,0.0000,0.0000,0.0000,0.0000
O,0.0000,0.0000,0.0000,0.0000,0.0000
B-_,0.0000,0.0000,0.0000,0.0000,0.0000
I-_,0.0000,0.0000,0.0000,0.0000,0.0000
I-_,0.0000,0.0000,0.0000,0.0000,0.0000


,O+and,O+find,B-_+the,I-_+black,I-_+pen
O,0.0000,0.0000,0.0000,0.0000,0.0000
O,0.0000,0.0000,0.0000,0.0000,0.0000
Phenomenon,0.0000,0.0000,0.0000,0.0000,0.0000
Phenomenon,0.0000,0.0000,0.0000,0.0000,0.0000
Phenomenon,0.0000,0.0000,0.0000,0.0000,0.0000


,and,find,the,black,pen
O,0.0046,0.9614,0.0118,0.0120,0.0102
O,0.0034,0.9856,0.0028,0.0035,0.0046
B-Phenomenon,0.0009,0.5119,0.2335,0.2014,0.0524
I-Phenomenon,0.0003,0.0479,0.0736,0.1232,0.7550
I-Phenomenon,0.0003,0.0325,0.0653,0.1001,0.8019


In [5]:
data_weighted_emb = load_json(FOLDER_2_BI_RNN)
display_all(data_weighted_emb)

true: Inspecting pred: Inspecting


words,check,the,toilet,paper
lexical_unit,1.0000,0.0000,0.0000,0.0000
attention_intent,0.0002,0.9318,0.0517,0.0162
slots_true,O,B-Ground,I-Ground,I-Ground
slots_pred,O,B-Phenomenon,I-Phenomenon,I-Containing_object


,check,the,toilet,paper
O,0.0000,0.0000,0.0000,0.0000
B-_,0.0000,0.0000,0.0000,0.0000
I-_,0.0000,0.0000,0.0000,0.0000
I-_,0.0000,0.0000,0.0000,0.0000


,O+check,B-_+the,I-_+toilet,I-_+paper
O,0.0000,0.0000,0.0000,0.0000
Phenomenon,0.0000,0.0000,0.0000,0.0000
Phenomenon,0.0000,0.0000,0.0000,0.0000
Containing_object,0.0000,0.0000,0.0000,0.0000


,check,the,toilet,paper
O,0.9959,0.0018,0.0017,0.0007
B-Phenomenon,0.9107,0.0280,0.0359,0.0255
I-Phenomenon,0.6533,0.1030,0.1203,0.1234
I-Containing_object,0.3390,0.1913,0.1409,0.3288


...

true: Searching pred: Searching


words,and,find,the,black,pen
lexical_unit,0.0000,1.0000,0.0000,0.0000,0.0000
attention_intent,0.0000,0.0012,0.6520,0.3448,0.0020
slots_true,O,O,B-Phenomenon,I-Phenomenon,I-Phenomenon
slots_pred,O,O,B-Phenomenon,I-Phenomenon,I-Phenomenon


,and,find,the,black,pen
O,0.0000,0.0000,0.0000,0.0000,0.0000
O,0.0000,0.0000,0.0000,0.0000,0.0000
B-_,0.0000,0.0000,0.0000,0.0000,0.0000
I-_,0.0000,0.0000,0.0000,0.0000,0.0000
I-_,0.0000,0.0000,0.0000,0.0000,0.0000


,O+and,O+find,B-_+the,I-_+black,I-_+pen
O,0.0000,0.0000,0.0000,0.0000,0.0000
O,0.0000,0.0000,0.0000,0.0000,0.0000
Phenomenon,0.0000,0.0000,0.0000,0.0000,0.0000
Phenomenon,0.0000,0.0000,0.0000,0.0000,0.0000
Phenomenon,0.0000,0.0000,0.0000,0.0000,0.0000


,and,find,the,black,pen
O,0.0211,0.9030,0.0549,0.0169,0.0042
O,0.0064,0.9000,0.0680,0.0234,0.0023
B-Phenomenon,0.0023,0.7093,0.2289,0.0508,0.0086
I-Phenomenon,0.0044,0.4162,0.3153,0.1857,0.0785
I-Phenomenon,0.0012,0.0843,0.2324,0.2828,0.3993
